In [1]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [2]:
import os
import sys
sys.path.append(os.path.abspath('..'))

from constants import (
    TARGET_SPARSITY_LOW, TARGET_SPARSITY_MID, TARGET_SPARSITY_HIGH,
    BATCH_SIZE_CNN, BATCH_SIZE_VIT, BATCH_SIZE_LLM,
    EPOCHS_SMALL_MODEL, EPOCHS_LARGE_MODEL, EPOCHS_VIT
)
from utils import get_device, get_num_workers, load_weights, print_statistics
from unstructured_pruning import check_model_sparsity, check_sparsity_distribution
from trainer import TrainingArguments, Trainer
from bacp import BaCPTrainingArguments, BaCPTrainer

from datasets.utils.logging import disable_progress_bar
disable_progress_bar()
os.environ["HF_DATASETS_CACHE"] = "/dbfs/hf_datasets"
os.environ["TOKENIZERS_PARALLELISM"] = "false" 


/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DEVICE = get_device()
NUM_WORKERS = get_num_workers()
print("Using device:", DEVICE)
print("Using", NUM_WORKERS, "workers")

Using device: cuda
Using 288 workers


In [4]:
# Notebook specific variables
MODEL_NAME = 'vgg19'
MODEL_TASK = 'cifar100'
TRAIN = True

## Baseline Accuracy

In [5]:
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    scheduler_type='linear_with_warmup',
    epochs=100,
    learning_type="baseline",
    patience=50,
    db=False,
)
trainer = Trainer(training_args=training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)


100%|██████████| 169M/169M [00:04<00:00, 38.7MB/s] 


[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Linear scheduler initialized with warmup steps: 830 and total steps: 8300
[TRAINER] Pruning not initialized
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/baseline/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Epoch [1/100]: Avg Loss: 4.5764 | Avg Accuracy: 11.27 | Model Sparsity: 0.0

[TRAINER] weights saved!


Epoch [2/100]: Avg Loss: 3.2725 | Avg Accuracy: 34.51 | Model Sparsity: 0.0

[TRAINER] weights saved!


...

Epoch [97/100]: Avg Loss: 0.0079 | Avg Accuracy: 68.18 | Model Sparsity: 0.0



Epoch [98/100]: Avg Loss: 0.0088 | Avg Accuracy: 68.01 | Model Sparsity: 0.0



Epoch [99/100]: Avg Loss: 0.0081 | Avg Accuracy: 68.50 | Model Sparsity: 0.0



Epoch [100/100]: Avg Loss: 0.0082 | Avg Accuracy: 68.12 | Model Sparsity: 0.0

[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.0



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     69.54%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.0000 (0.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        baseline
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               100

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



## Pruning Accuracy

### Magnitude Pruning

In [6]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.95_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/pruning/magnitude_pruning/0.95/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Epoch [1/5]: Avg Loss: 0.1169 | Avg Accuracy: 64.43 | Model Sparsity: 0.4636

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.1686 | Avg Accuracy: 64.34 | Model Sparsity: 0.4636



...

Recovery epoch [7/10]: Avg Loss: 0.0824 | Avg Accuracy: 66.18 | Model Sparsity: 0.95



Recovery epoch [8/10]: Avg Loss: 0.0824 | Avg Accuracy: 66.02 | Model Sparsity: 0.95



Recovery epoch [9/10]: Avg Loss: 0.0813 | Avg Accuracy: 65.44 | Model Sparsity: 0.95



Recovery epoch [10/10]: Avg Loss: 0.0768 | Avg Accuracy: 66.02 | Model Sparsity: 0.95

[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     68.40%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [8]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.97_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/pruning/magnitude_pruning/0.97/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Epoch [1/5]: Avg Loss: 0.1163 | Avg Accuracy: 64.76 | Model Sparsity: 0.4734

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.1632 | Avg Accuracy: 64.73 | Model Sparsity: 0.4734



...

Recovery epoch [7/10]: Avg Loss: 0.1441 | Avg Accuracy: 66.10 | Model Sparsity: 0.97



Recovery epoch [8/10]: Avg Loss: 0.1438 | Avg Accuracy: 66.04 | Model Sparsity: 0.97



Recovery epoch [9/10]: Avg Loss: 0.1360 | Avg Accuracy: 66.20 | Model Sparsity: 0.97



Recovery epoch [10/10]: Avg Loss: 0.1304 | Avg Accuracy: 66.06 | Model Sparsity: 0.97

[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     67.45%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [9]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.99
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.99_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/pruning/magnitude_pruning/0.99/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Epoch [1/5]: Avg Loss: 0.1303 | Avg Accuracy: 64.34 | Model Sparsity: 0.4831

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.1610 | Avg Accuracy: 65.40 | Model Sparsity: 0.4831

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 0.6944 | Avg Accuracy: 63.83 | Model Sparsity: 0.99



Recovery epoch [8/10]: Avg Loss: 0.6746 | Avg Accuracy: 63.31 | Model Sparsity: 0.99



Recovery epoch [9/10]: Avg Loss: 0.6520 | Avg Accuracy: 64.37 | Model Sparsity: 0.99



Recovery epoch [10/10]: Avg Loss: 0.6469 | Avg Accuracy: 64.38 | Model Sparsity: 0.99

[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     66.86%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



### Movement Pruning

In [10]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_snip_pruning_0.95_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/pruning/snip_pruning/0.95/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Epoch [1/5]: Avg Loss: 0.1898 | Avg Accuracy: 63.77 | Model Sparsity: 0.4636

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.2277 | Avg Accuracy: 64.47 | Model Sparsity: 0.4636

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 0.1496 | Avg Accuracy: 66.21 | Model Sparsity: 0.95



Recovery epoch [8/10]: Avg Loss: 0.1381 | Avg Accuracy: 65.95 | Model Sparsity: 0.95



Recovery epoch [9/10]: Avg Loss: 0.1405 | Avg Accuracy: 66.48 | Model Sparsity: 0.95

[TRAINER] weights saved!


Recovery epoch [10/10]: Avg Loss: 0.1348 | Avg Accuracy: 65.43 | Model Sparsity: 0.95

[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_snip_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     67.27%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [11]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_snip_pruning_0.97_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/pruning/snip_pruning/0.97/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Epoch [1/5]: Avg Loss: 0.1770 | Avg Accuracy: 64.44 | Model Sparsity: 0.4734

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.2525 | Avg Accuracy: 64.90 | Model Sparsity: 0.4734

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 0.2769 | Avg Accuracy: 65.64 | Model Sparsity: 0.97



Recovery epoch [8/10]: Avg Loss: 0.2709 | Avg Accuracy: 65.00 | Model Sparsity: 0.97



Recovery epoch [9/10]: Avg Loss: 0.2487 | Avg Accuracy: 65.21 | Model Sparsity: 0.97



Recovery epoch [10/10]: Avg Loss: 0.2301 | Avg Accuracy: 65.68 | Model Sparsity: 0.97

[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_snip_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     67.03%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [12]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.99
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_snip_pruning_0.99_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/pruning/snip_pruning/0.99/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Epoch [1/5]: Avg Loss: 0.1906 | Avg Accuracy: 64.13 | Model Sparsity: 0.4831

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.2382 | Avg Accuracy: 64.26 | Model Sparsity: 0.4831

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 4.0031 | Avg Accuracy: 3.53 | Model Sparsity: 0.99



Recovery epoch [8/10]: Avg Loss: 3.9877 | Avg Accuracy: 4.06 | Model Sparsity: 0.99

[TRAINER] weights saved!


Recovery epoch [9/10]: Avg Loss: 3.9538 | Avg Accuracy: 4.52 | Model Sparsity: 0.99

[TRAINER] weights saved!


Recovery epoch [10/10]: Avg Loss: 3.9396 | Avg Accuracy: 4.59 | Model Sparsity: 0.99

[TRAINER] weights saved!
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_snip_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     4.18%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



### Wanda Pruning

In [13]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: wanda_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_wanda_pruning_0.95_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/pruning/wanda_pruning/0.95/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0
[Pruner] Adding hooks


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Training Epoch [1/5]:   2%|▏         | 2/83 [00:03<01:46,  1.32s/it, Loss=0.5680, Sparsity=0.4636]


[Pruner] Removing hooks


...

Recovery epoch [7/10]: Avg Loss: 0.0966 | Avg Accuracy: 65.95 | Model Sparsity: 0.95



Recovery epoch [8/10]: Avg Loss: 0.0964 | Avg Accuracy: 66.14 | Model Sparsity: 0.95



Recovery epoch [9/10]: Avg Loss: 0.0893 | Avg Accuracy: 66.64 | Model Sparsity: 0.95

[TRAINER] weights saved!


Recovery epoch [10/10]: Avg Loss: 0.0882 | Avg Accuracy: 65.23 | Model Sparsity: 0.95

[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_wanda_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     66.58%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         wanda_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [14]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: wanda_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_wanda_pruning_0.97_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/pruning/wanda_pruning/0.97/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0
[Pruner] Adding hooks


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Training Epoch [1/5]:   4%|▎         | 3/83 [00:02<00:47,  1.69it/s, Loss=0.6916, Sparsity=0.4734]


[Pruner] Removing hooks


...

Recovery epoch [7/10]: Avg Loss: 0.1322 | Avg Accuracy: 65.61 | Model Sparsity: 0.97



Recovery epoch [8/10]: Avg Loss: 0.1204 | Avg Accuracy: 66.17 | Model Sparsity: 0.97



Recovery epoch [9/10]: Avg Loss: 0.1172 | Avg Accuracy: 65.57 | Model Sparsity: 0.97



Recovery epoch [10/10]: Avg Loss: 0.1194 | Avg Accuracy: 65.96 | Model Sparsity: 0.97

[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_wanda_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     67.41%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         wanda_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [15]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: wanda_pruning
[TRAINER] Target sparsity: 0.99
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_wanda_pruning_0.99_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/pruning/wanda_pruning/0.99/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0
[Pruner] Adding hooks


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Training Epoch [1/5]:   1%|          | 1/83 [00:03<03:58,  2.91s/it, Loss=0.8471, Sparsity=0.4831]


[Pruner] Removing hooks


...

Recovery epoch [7/10]: Avg Loss: 0.6127 | Avg Accuracy: 64.08 | Model Sparsity: 0.99

[TRAINER] weights saved!


Recovery epoch [8/10]: Avg Loss: 0.6029 | Avg Accuracy: 64.26 | Model Sparsity: 0.99

[TRAINER] weights saved!


Recovery epoch [9/10]: Avg Loss: 0.5700 | Avg Accuracy: 63.84 | Model Sparsity: 0.99



Recovery epoch [10/10]: Avg Loss: 0.5544 | Avg Accuracy: 64.72 | Model Sparsity: 0.99

[TRAINER] weights saved!
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_wanda_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     65.82%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         wanda_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



## BaCP Accuracies

### Magnitude Pruning

In [16]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='magnitude_pruning',
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[ERROR] Could not load weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[ERROR] Attempting partial load
[TRAINER] Weights loaded successfully
[TRAINER] Initialized BaCP models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.1)
[TRAINER] No scheduler initialized
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.95_bacp_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/bacp_pruning/magnitude_pruning/0.95/run_1.log


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Epoch [1/5]: Avg Total Loss: 8.3331 | Avg PrC Loss: 3.6471 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 3.5383 | Avg CE Loss: 1.1476 | Model Sparsity: 0.4636

[BaCP] weights saved!


Retraining Epoch [1/10]: Avg Total Loss: 7.8921 | Avg PrC Loss: 3.4766 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 3.3943 | Avg CE Loss: 1.0212 | Model Sparsity: 0.4636



...

Epoch [47/50]: Avg Loss: 0.3087 | Avg Accuracy: 65.95 | Model Sparsity: 0.95



Epoch [48/50]: Avg Loss: 0.2954 | Avg Accuracy: 66.32 | Model Sparsity: 0.95



Epoch [49/50]: Avg Loss: 0.2912 | Avg Accuracy: 66.21 | Model Sparsity: 0.95



Epoch [50/50]: Avg Loss: 0.2730 | Avg Accuracy: 65.47 | Model Sparsity: 0.95

[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.95_bacp_finetune.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     64.83%

Model Information:
------------------------------
  Total Parameters:     139,979,940
  Trainable Parameters: 139,979,940
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar100
  Learning Type:        bacp_finetune
  Batch Size:           512
  Learning Rate:        0.0001
  Optimizer:            adamw
  Epochs:               50

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='magnitude_pruning',
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[ERROR] Could not load weights: ./research/vgg19/cifar100/vgg19_cifar100_baseline.pt
[ERROR] Attempting partial load
[TRAINER] Weights loaded successfully
[TRAINER] Initialized BaCP models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.1)
[TRAINER] No scheduler initialized
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.97_bacp_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar100/bacp_pruning/magnitude_pruning/0.97/run_1.log


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Epoch [1/5]: Avg Total Loss: 8.6539 | Avg PrC Loss: 3.8233 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 3.6618 | Avg CE Loss: 1.1687 | Model Sparsity: 0.4734

[BaCP] weights saved!


Retraining Epoch [1/10]: Avg Total Loss: 8.3589 | Avg PrC Loss: 3.6413 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 3.5724 | Avg CE Loss: 1.1453 | Model Sparsity: 0.4734



...

Retraining Epoch [10/10]: Avg Total Loss: 18.7277 | Avg PrC Loss: 3.3959 | Avg SnC Loss: 11.3576 | Avg FiC Loss: 3.2707 | Avg CE Loss: 0.7035 | Model Sparsity: 0.97

[BaCP] weights saved!
[BaCP TRAINER] Mask generated from current model.
[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.97_bacp_pruning.pt
[ERROR] Could not load weights: ./research/vgg19/cifar100/vgg19_cifar100_magnitude_pruning_0.97_bacp_pruning.pt
[ERROR] Attempting partial load
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 0.0001)
[CV DATALOADERS] Loaded cifar100 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar100
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Finetuning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Current sparsity: 0.9700
[TRAINER] Saving model to: ./research/

Epoch [1/50]: Avg Loss: 2.9712 | Avg Accuracy: 48.37 | Model Sparsity: 0.97

[TRAINER] weights saved!


Epoch [2/50]: Avg Loss: 1.9621 | Avg Accuracy: 52.22 | Model Sparsity: 0.97

[TRAINER] weights saved!


Epoch [3/50]: Avg Loss: 1.7945 | Avg Accuracy: 53.40 | Model Sparsity: 0.97

[TRAINER] weights saved!


Training Epoch [4/50]:   0%|          | 0/83 [00:00<?, ?it/s]

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='magnitude_pruning',
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

### SNIP-it Pruning

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='snip_pruning',
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='snip_pruning',
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='snip_pruning',
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

### Wanda Pruning

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='wanda_pruning',
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='wanda_pruning',
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='wanda_pruning',
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)